<a href="https://colab.research.google.com/github/adrian-ja-projects/train-prediction-project/blob/fea_data_analysis/data_analysis_train_prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/adrian-ja-projects/train-prediction-project.git

Cloning into 'train-prediction-project'...
remote: Enumerating objects: 83, done.
remote: Counting objects: 100% (83/83), done.
remote: Compressing objects: 100% (81/81), done.
remote: Total 83 (delta 58), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (83/83), done.


In [2]:
%run /content/train-prediction-project/main_notebook.ipynb

INFO: Creating list for extraction loop...
INFO: Extraction dates are between 2020-01-01 and 2020-04-09
INFO: Starting extraction...
INFO: Extraction completed a total of 100 file were successfully extracted into the staging area
INFO: Installing Spark dependencies...
INFO: Spark dependencies installed
INFO: env variables created
INFO: Starting transformation of json files...
Data in raw ready to upload to a db
INFO: table 27_travel_duration has been overwriten on the path /content/uc_train_prediction/27_travel_duration. Table ready for analysis


In [3]:
from pyspark.sql import functions as F

In [4]:
#create spark session
spark = SparkSession.builder\
        .master("local")\
        .appName("data_analysis")\
        .config('spark.ui.port', '4050')\
        .getOrCreate()

In [5]:
uc_file_path = "/content/uc_train_prediction/27_travel_duration/"
df_27_travel_duration = spark.read.format("parquet").load(uc_file_path)

In [6]:
train_travel_avg_time = df_27_travel_duration.select(F.avg("travelDurationMinutes")).collect()[0][0]

In [7]:
def calculate_suggesting_departure_time(full_travel_time: int, departing_hour: int, departing_minute: int)-> str:
  """
  Method to calculate the suggested upper limit for the departure time of the IC27 train
  """
  return datetime.timedelta(hours=16, minutes=15) - datetime.timedelta(minutes=full_travel_time)

In [8]:
walking_time = 15
full_travel_time = train_travel_avg_time + walking_time

suggested_departure_time = calculate_suggesting_departure_time(full_travel_time, 16, 15)
print(f"If the IC27 train is scheduled to leave after {suggested_departure_time}, we strongly suggest to take an earlier train")

If the IC27 train is scheduled to leave after 14:25:22.330000, we strongly suggest to take an earlier train
